<a href="https://colab.research.google.com/github/miltiadiss/Big-Data-Management-Systems/blob/main/big_data_question_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def vehicles_to_pandas():
    # Δημιουργία πλασματικών δεδομένων για την εξομοίωση
    data = {
        'name': ['vehicle_1', 'vehicle_2'],
        'dn': [1, 1],
        'orig': ['A', 'B'],
        'dest': ['C', 'D'],
        't': [15, 25],
        'link': ['link_1', 'link_2'],
        'x': [100, 200],
        's': [10, 20],
        'v': [50, 60]
    }
    return pd.DataFrame(data)

# Παράγει τα δεδομένα με τη μορφή pandas DataFrame
data = vehicles_to_pandas()

# Εμφάνιση των δεδομένων
print(data)

# Αποθήκευση των δεδομένων σε αρχείο CSV
data.to_csv('vehicle_data.csv', index=False)

        name  dn orig dest   t    link    x   s   v
0  vehicle_1   1    A    C  15  link_1  100  10  50
1  vehicle_2   1    B    D  25  link_2  200  20  60


In [ ]:
from kafka import KafkaProducer
import json
import time
from datetime import datetime

# Φόρτωση δεδομένων
data = pd.read_csv('vehicle_data.csv')

# Αρχικοποίηση Kafka producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Χρόνος εκκίνησης του producer
start_time = datetime.now()

# Συνάρτηση για αποστολή δεδομένων στο Kafka
def send_vehicle_data():
    current_time = datetime.now()
    elapsed_time = (current_time - start_time).total_seconds()

    for index, row in data.iterrows():
        if row['t'] <= elapsed_time:
            if row['link'] != 'waiting at origin node':
                vehicle_data = {
                    'name': row['name'],
                    'dn': row['dn'],
                    'orig': row['orig'],
                    'dest': row['dest'],
                    't': row['t'],
                    'link': row['link'],
                    'x': row['x'],
                    's': row['s'],
                    'v': row['v'],
                    'timestamp': current_time.isoformat()
                }
                producer.send('vehicle_positions', vehicle_data)
                print(f"Sent data: {vehicle_data}")

# Περιοδική αποστολή δεδομένων
try:
    while True:
        send_vehicle_data()
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopping producer.")
finally:
    producer.close()

In [ ]:
from kafka import KafkaConsumer

# Αρχικοποίηση Kafka consumer
consumer = KafkaConsumer(
    'vehicle_positions',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

print("Starting consumer...")
for message in consumer:
    vehicle_data = message.value
    print(f"Received data: {vehicle_data}")